# Amazon Fine Foods Review: 
 
 this covers Loading the data from SQL,Data cleaning using RE(Regular exp),
 text preprocessing usin NLTK- tokenization,
 Stemming,Lemmitization,BOW(Bag of words),unigrams/bigrams,
 TF-IDF,Word2vec,Avg word2vec,TF-IDF word2vec

# Objective:
Given a review, determine whether the review is positive (Rating of 4 or 5) or negative (rating of 1 or 2).

In [ ]:
%matplotlib inline
import warnings
warnings.filterwarnings("ignore")

import sqlite3
import pandas as pd
import numpy as np
import nltk
import string
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.feature_extraction.text import TfidfVectorizer

from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics import confusion_matrix
from sklearn import metrics
from sklearn.metrics import roc_curve, auc

import re
# Tutorial about Python regular expressions: https://pymotw.com/2/re/
import string
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer
from nltk.stem.wordnet import WordNetLemmatizer

from gensim.models import Word2Vec
from gensim.models import KeyedVectors
import pickle

from tqdm import tqdm
import os

In [ ]:
# using the SQLite Table to read data.
con = sqlite3.connect('./amazon-fine-food-reviews/database.sqlite') 
#filtering only positive and negative reviews i.e. 
# not taking into consideration those reviews with Score=3
filtered_data = pd.read_sql_query('SELECT * FROM Reviews WHERE Score != 3 LIMIT 5000', con) 
filtered_data.head()

,Id,ProductId,UserId,ProfileName,HelpfulnessNumerator,HelpfulnessDenominator,Score,Time,Summary,Text
0,1,B001E4KFG0,A3SGXH7AUHU8GW,delmartian,1,1,5,1303862400,Good Quality Dog Food,I have bought several of the Vitality canned d...
1,2,B00813GRG4,A1D87F6ZCVE5NK,dll pa,0,0,1,1346976000,Not as Advertised,Product arrived labeled as Jumbo Salted Peanut...
2,3,B000LQOCH0,ABXLMWJIXXAIN,"Natalia Corres ""Natalia Corres""",1,1,4,1219017600,"""Delight"" says it all",This is a confection that has been around a fe...
3,4,B000UA0QIQ,A395BORC6FGVXV,Karl,3,3,2,1307923200,Cough Medicine,If you are looking for the secret ingredient i...
4,5,B006K2ZZ7K,A1UQRSCLF8GW1T,"Michael D. Bigham ""M. Wassir""",0,0,5,1350777600,Great taffy,Great taffy at a great price. There was a wid...


In [ ]:
filtered_data['Score'] = filtered_data['Score'].apply(lambda x: 1 if x>3 else 0)

In [ ]:
#Sorting data according to ProductId in ascending order
sorted_data=filtered_data.sort_values('ProductId', axis=0, ascending=True, 
                                      inplace=False, kind='quicksort', na_position='last')

In [ ]:
#Deduplication of entries
final=sorted_data.drop_duplicates(subset={"UserId","ProfileName","Time","Text"}, 
                                  keep='first', inplace=False)
final.shape

(4986, 10)

In [ ]:
#Checking to see how much % of data still remains
(final['Id'].size*1.0)/(filtered_data['Id'].size*1.0)*100

99.72

In [ ]:
final=final[final.HelpfulnessNumerator<=final.HelpfulnessDenominator]

In [ ]:
final.shape

(4986, 10)

In [ ]:
final.Score.value_counts()

1    4178
0     808
Name: Score, dtype: int64

# Text Preprocessing:
Hence in the Preprocessing phase we do the following in the order below:-

1.Begin by removing the html tags
2.Remove any punctuations or limited set of special characters like , or . or # etc.
3.Check if the word is made up of english letters and is not alpha-numeric
4.Check to see if the length of the word is greater than 2 (as it was researched that there is no adjective in 2-letters)
5.Convert the word to lowercase
6.Remove Stopwords
7.Finally Snowball Stemming the word (it was obsereved to be better than Porter Stemming)
8.After which we collect the words used to describe positive and negative reviews

In [ ]:
# printing some random reviews
sent_0 = final['Text'].values[0]
print(sent_0)
print("="*50)

sent_1000 = final['Text'].values[1000]
print(sent_1000)
print("="*50)

sent_1500 = final['Text'].values[1500]
print(sent_1500)
print("="*50)

sent_4900 = final['Text'].values[4900]
print(sent_4900)
print("="*50)

Why is this $[...] when the same product is available for $[...] here?<br />http://www.amazon.com/VICTOR-FLY-MAGNET-BAIT-REFILL/dp/B00004RBDY<br /><br />The Victor M380 and M502 traps are unreal, of course -- total fly genocide. Pretty stinky, but only right nearby.
I recently tried this flavor/brand and was surprised at how delicious these chips are.  The best thing was that there were a lot of "brown" chips in the bsg (my favorite), so I bought some more through amazon and shared with family and friends.  I am a little disappointed that there are not, so far, very many brown chips in these bags, but the flavor is still very good.  I like them better than the yogurt and green onion flavor because they do not seem to be as salty, and the onion flavor is better.  If you haven't eaten Kettle chips before, I recommend that you try a bag before buying bulk.  They are thicker and crunchier than Lays but just as fresh out of the bag.
Wow.  So far, two two-star reviews.  One obviously had no 

In [ ]:
# remove tags,URL's / clean data

# remove urls from text python: https://stackoverflow.com/a/40823105/4084039

sent_0  = re.sub(r"http\S+","",sent_0)
sent_1000 = re.sub(r"http\S+", "", sent_1000)
sent_150 = re.sub(r"http\S+", "", sent_1500)
sent_4900 = re.sub(r"http\S+", "", sent_4900)
print(sent_0)

Why is this $[...] when the same product is available for $[...] here?<br /> /><br />The Victor M380 and M502 traps are unreal, of course -- total fly genocide. Pretty stinky, but only right nearby.


In [ ]:
# https://stackoverflow.com/questions/16206380/python-beautifulsoup-how-to-remove-all-tags-from-an-element
from bs4 import BeautifulSoup

soup = BeautifulSoup(sent_0, 'lxml')
text = soup.get_text()
print(text)
print("="*50)

soup = BeautifulSoup(sent_1000, 'lxml')
text = soup.get_text()
print(text)
print("="*50)

soup = BeautifulSoup(sent_1500, 'lxml')
text = soup.get_text()
print(text)
print("="*50)

soup = BeautifulSoup(sent_4900, 'lxml')
text = soup.get_text()
print(text)

Why is this $[...] when the same product is available for $[...] here? />The Victor M380 and M502 traps are unreal, of course -- total fly genocide. Pretty stinky, but only right nearby.
I recently tried this flavor/brand and was surprised at how delicious these chips are.  The best thing was that there were a lot of "brown" chips in the bsg (my favorite), so I bought some more through amazon and shared with family and friends.  I am a little disappointed that there are not, so far, very many brown chips in these bags, but the flavor is still very good.  I like them better than the yogurt and green onion flavor because they do not seem to be as salty, and the onion flavor is better.  If you haven't eaten Kettle chips before, I recommend that you try a bag before buying bulk.  They are thicker and crunchier than Lays but just as fresh out of the bag.
Wow.  So far, two two-star reviews.  One obviously had no idea what they were ordering; the other wants crispy cookies.  Hey, I'm sorry; b

In [ ]:
# https://stackoverflow.com/a/47091490/4084039
import re

def decontracted(phrase):
    # specific
    phrase = re.sub(r"won't", "will not", phrase)
    phrase = re.sub(r"can\'t", "can not", phrase)
    phrase = re.sub(r"haven't", "have not", phrase)

    # general
    phrase = re.sub(r"n\'t", " not", phrase)
    phrase = re.sub(r"\'re", " are", phrase)
    phrase = re.sub(r"\'s", " is", phrase)
    phrase = re.sub(r"\'d", " would", phrase)
    phrase = re.sub(r"\'ll", " will", phrase)
    phrase = re.sub(r"\'t", " not", phrase)
    phrase = re.sub(r"\'ve", " have", phrase)
    phrase = re.sub(r"\'m", " am", phrase)
    return phrase

In [ ]:
sent_1500 = decontracted(sent_1000)
print(sent_1500)
print("="*50)

I recently tried this flavor/brand and was surprised at how delicious these chips are.  The best thing was that there were a lot of "brown" chips in the bsg (my favorite), so I bought some more through amazon and shared with family and friends.  I am a little disappointed that there are not, so far, very many brown chips in these bags, but the flavor is still very good.  I like them better than the yogurt and green onion flavor because they do not seem to be as salty, and the onion flavor is better.  If you have not eaten Kettle chips before, I recommend that you try a bag before buying bulk.  They are thicker and crunchier than Lays but just as fresh out of the bag.


In [ ]:
#remove words with numbers python: https://stackoverflow.com/a/18082370/4084039
sent_0 = re.sub("\S*\d\S*", "", sent_0).strip()
print(sent_0)

Why is this $[...] when the same product is available for $[...] here?<br /> /><br />The Victor  and  traps are unreal, of course -- total fly genocide. Pretty stinky, but only right nearby.


In [ ]:
#remove spacial character: https://stackoverflow.com/a/5843547/4084039
sent_1500 = re.sub('[^A-Za-z0-9]+', ' ', sent_1500)
print(sent_1500)

I recently tried this flavor brand and was surprised at how delicious these chips are The best thing was that there were a lot of brown chips in the bsg my favorite so I bought some more through amazon and shared with family and friends I am a little disappointed that there are not so far very many brown chips in these bags but the flavor is still very good I like them better than the yogurt and green onion flavor because they do not seem to be as salty and the onion flavor is better If you have not eaten Kettle chips before I recommend that you try a bag before buying bulk They are thicker and crunchier than Lays but just as fresh out of the bag 


In [ ]:
# https://gist.github.com/sebleier/554280
# we are removing the words from the stop words list: 'no', 'nor', 'not'
# <br /><br /> ==> after the above steps, we are getting "br br"
# we are including them into stop words list
# instead of <br /> if we have <br/> these tags would have revmoved in the 1st step

stopwords= set(['br', 'the', 'i', 'me', 'my', 'myself', 'we', 'our', 'ours', 'ourselves', 'you', "you're", "you've",\
            "you'll", "you'd", 'your', 'yours', 'yourself', 'yourselves', 'he', 'him', 'his', 'himself', \
            'she', "she's", 'her', 'hers', 'herself', 'it', "it's", 'its', 'itself', 'they', 'them', 'their',\
            'theirs', 'themselves', 'what', 'which', 'who', 'whom', 'this', 'that', "that'll", 'these', 'those', \
            'am', 'is', 'are', 'was', 'were', 'be', 'been', 'being', 'have', 'has', 'had', 'having', 'do', 'does', \
            'did', 'doing', 'a', 'an', 'the', 'and', 'but', 'if', 'or', 'because', 'as', 'until', 'while', 'of', \
            'at', 'by', 'for', 'with', 'about', 'against', 'between', 'into', 'through', 'during', 'before', 'after',\
            'above', 'below', 'to', 'from', 'up', 'down', 'in', 'out', 'on', 'off', 'over', 'under', 'again', 'further',\
            'then', 'once', 'here', 'there', 'when', 'where', 'why', 'how', 'all', 'any', 'both', 'each', 'few', 'more',\
            'most', 'other', 'some', 'such', 'only', 'own', 'same', 'so', 'than', 'too', 'very', \
            's', 't', 'can', 'will', 'just', 'don', "don't", 'should', "should've", 'now', 'd', 'll', 'm', 'o', 're', \
            've', 'y', 'ain', 'aren', "aren't", 'couldn', "couldn't", 'didn', "didn't", 'doesn', "doesn't", 'hadn',\
            "hadn't", 'hasn', "hasn't", 'haven', "haven't", 'isn', "isn't", 'ma', 'mightn', "mightn't", 'mustn',\
            "mustn't", 'needn', "needn't", 'shan', "shan't", 'shouldn', "shouldn't", 'wasn', "wasn't", 'weren', "weren't", \
            'won', "won't", 'wouldn', "wouldn't"])

In [ ]:
# Combining all the above stundents 
from tqdm import tqdm
preprocessed_reviews = []
# tqdm is for printing the status bar
for sentance in tqdm(final['Text'].values):
    sentance = re.sub(r"http\S+", "", sentance)
    sentance = BeautifulSoup(sentance, 'lxml').get_text()
    sentance = decontracted(sentance)
    sentance = re.sub("\S*\d\S*", "", sentance).strip()
    sentance = re.sub('[^A-Za-z]+', ' ', sentance)
    # https://gist.github.com/sebleier/554280
    sentance = ' '.join(e.lower() for e in sentance.split() if e.lower() not in stopwords)
    preprocessed_reviews.append(sentance.strip())

100%|████████████████████████████████████████████████████████████████████████████| 4986/4986 [00:03<00:00, 1637.52it/s]


In [ ]:
preprocessed_reviews[1500]

'wow far two two star reviews one obviously no idea ordering wants crispy cookies hey sorry reviews nobody good beyond reminding us look ordering chocolate oatmeal cookies not like combination not order type cookie find combo quite nice really oatmeal sort calms rich chocolate flavor gives cookie sort coconut type consistency let also remember tastes differ given opinion soft chewy cookies advertised not crispy cookies blurb would say crispy rather chewy happen like raw cookie dough however not see taste like raw cookie dough soft however confusion yes stick together soft cookies tend not individually wrapped would add cost oh yeah chocolate chip cookies tend somewhat sweet want something hard crisp suggest nabiso ginger snaps want cookie soft chewy tastes like combination chocolate oatmeal give try place second order'

In [ ]:
final['Summary'].head()

2546                     thirty bucks?
2547                      Flies Begone
1145    WOW Make your own 'slickers' !
1146                     Great Product
2942                       Good stuff!
Name: Summary, dtype: object

In [ ]:
# Combining all the above stundents 
from tqdm import tqdm
preprocessed_summary = []
# tqdm is for printing the status bar
for sentance in tqdm(final['Summary'].values):
    sentance = re.sub(r"http\S+", "", sentance)
    sentance = BeautifulSoup(sentance, 'lxml').get_text()
    sentance = decontracted(sentance)
    sentance = re.sub("\S*\d\S*", "", sentance).strip()
    sentance = re.sub('[^A-Za-z]+', ' ', sentance)
    # https://gist.github.com/sebleier/554280
    sentance = ' '.join(e.lower() for e in sentance.split() if e.lower() not in stopwords)
    preprocessed_summary.append(sentance.strip())

100%|████████████████████████████████████████████████████████████████████████████| 4986/4986 [00:01<00:00, 2965.28it/s]


In [ ]:
preprocessed_summary[317]

'great gift'

In [ ]:
final.loc[final['Summary'] == "Good stuff!"]

,Id,ProductId,UserId,ProfileName,HelpfulnessNumerator,HelpfulnessDenominator,Score,Time,Summary,Text
2942,3204,B000084DVR,A1UGDJP1ZJWVPF,"T. Moore ""thoughtful reader""",1,1,1,1177977600,Good stuff!,I'm glad my 45lb cocker/standard poodle puppy ...
317,349,B00067AD4U,A3GT3KCGY9EYF4,Siri J,1,1,1,1198195200,Good stuff!,"""These are delicious! The chocolate is excelle..."


# BAG OF WORDS

In [ ]:
# Converts text to numeric vector
# takes all unique words from text column in all the documents.. 
# that will be the no of dimensions of the vector 
# for each document,compute no of repetetions of each word ie.. [0,0,5,2,1]
# it is sparse matrix with any 0's
# binary BOW or Boolean BOW : if word is present.. put 1 else 0, ie.. [0,0,1,1,1]
# Comparing words: Tasty and delicious - 
# If words are more similar, then distance between them will be less
# with BOW,BBOW, TF-IDF, semantic analysis cant be done

In [ ]:
count_vect = CountVectorizer()
count_vect.fit(preprocessed_reviews)
print('some Feature names:',count_vect.get_feature_names()[0:10])
print('*'*50)
final_counts = count_vect.transform(preprocessed_reviews)
print("the type of count vectorizer ",type(final_counts))
print("the shape of out text BOW vectorizer ",final_counts.get_shape())
print("the number of unique words ", final_counts.get_shape()[1])

some Feature names: ['aa', 'aahhhs', 'aback', 'abandon', 'abates', 'abbott', 'abby', 'abdominal', 'abiding', 'ability']
**************************************************
the type of count vectorizer  <class 'scipy.sparse.csr.csr_matrix'>
the shape of out text BOW vectorizer  (4986, 12997)
the number of unique words  12997


In [ ]:
print(final_counts[1].toarray())

[[0 0 0 ... 0 0 0]]


# Bi-Grams and n-Grams

In [ ]:
#bi-gram, tri-gram and n-gram

#removing stop words like "not" should be avoided before building n-grams
# count_vect = CountVectorizer(ngram_range=(1,2))
# please do read the CountVectorizer documentation
#http://scikit-learn.org/stable/modules/generated/sklearn.feature_extraction.text.CountVectorizer.html
# you can choose these numebrs min_df=10, max_features=5000, of your choice


count_vect = CountVectorizer(ngram_range=(1,2), min_df=10, max_features=5000)
final_bigram_counts = count_vect.fit_transform(preprocessed_reviews)
print("the type of count vectorizer ",type(final_bigram_counts))
print("the shape of out text BOW vectorizer ",final_bigram_counts.get_shape())
print("the number of unique words including both unigrams and bigrams ", final_bigram_counts.get_shape()[1])

the type of count vectorizer  <class 'scipy.sparse.csr.csr_matrix'>
the shape of out text BOW vectorizer  (4986, 3144)
the number of unique words including both unigrams and bigrams  3144


# TF-IDF

In [ ]:
# term freq - no of times a word occur in a document/total no of words in a document
# inverse doc freq - it gives no of times a word occurs in the entire corpus

In [ ]:
tf_idf_vect = TfidfVectorizer(ngram_range = (1,2),min_df = 10)
final_tf_idf = tf_idf_vect.fit_transform(preprocessed_reviews)
print('Features:',tf_idf_vect.get_feature_names()[0:10])
print('shape:',final_tf_idf.shape)
print('type:',type(final_tf_idf))

Features: ['ability', 'able', 'able find', 'able get', 'absolute', 'absolutely', 'absolutely delicious', 'absolutely love', 'absolutely no', 'according']
shape: (4986, 3144)
type: <class 'scipy.sparse.csr.csr_matrix'>


# Word2Vec

In [ ]:
# it considers semantic meaning.
# we need to give corpus, it will automatically create vector for each word
# Train your own Word2Vec model using your own text corpus
list_of_sentance = []
for sentance in preprocessed_reviews:
    list_of_sentance.append(sentance.split())

In [ ]:
list_of_sentance[2]

['received',
 'shipment',
 'could',
 'hardly',
 'wait',
 'try',
 'product',
 'love',
 'slickers',
 'call',
 'instead',
 'stickers',
 'removed',
 'easily',
 'daughter',
 'designed',
 'signs',
 'printed',
 'reverse',
 'use',
 'car',
 'windows',
 'printed',
 'beautifully',
 'print',
 'shop',
 'program',
 'going',
 'lot',
 'fun',
 'product',
 'windows',
 'everywhere',
 'surfaces',
 'like',
 'tv',
 'screens',
 'computer',
 'monitors']

In [ ]:
# Using Google News Word2Vectors

# in this project we are using a pretrained model by google
# its 3.3G file, once you load this into your memory 
# it occupies ~9Gb, so please do this step only if you have >12G of ram
# we will provide a pickle file wich contains a dict , 
# and it contains all our courpus words as keys and  model[word] as values
# To use this code-snippet, download "GoogleNews-vectors-negative300.bin" 
# from https://drive.google.com/file/d/0B7XkCwpI5KDYNlNUTTlSS21pQmM/edit
# it's 1.9GB in size.


# http://kavita-ganesan.com/gensim-word2vec-tutorial-starter-code/#.W17SRFAzZPY
# you can comment this whole cell
# or change these varible according to your need

is_your_ram_gt_16g=False
want_to_use_google_w2v = False
want_to_train_w2v = True

if want_to_train_w2v:
    # min_count = 5 considers only words that occured atleast 5 times
    w2v_model=Word2Vec(list_of_sentance,min_count=5,size=50, workers=4)
    print(w2v_model.wv.most_similar('great'))
    print('='*50)
    print(w2v_model.wv.most_similar('worst'))
    
elif want_to_use_google_w2v and is_your_ram_gt_16g:
    if os.path.isfile('GoogleNews-vectors-negative300.bin'):
        w2v_model=KeyedVectors.load_word2vec_format('GoogleNews-vectors-negative300.bin', binary=True)
        print(w2v_model.wv.most_similar('great'))
        print(w2v_model.wv.most_similar('worst'))
    else:
        print("you don't have gogole's word2vec file, keep want_to_train_w2v = True, to train your own w2v ")

[('snack', 0.9957177639007568), ('want', 0.9956516027450562), ('excellent', 0.9953365325927734), ('think', 0.9952346682548523), ('alternative', 0.9949909448623657), ('satisfying', 0.9947104454040527), ('care', 0.9946879744529724), ('chewy', 0.9946634769439697), ('feel', 0.9945921301841736), ('treat', 0.9945704340934753)]
[('wife', 0.9995176792144775), ('feels', 0.9994959235191345), ('part', 0.9994925856590271), ('popcorn', 0.9994866847991943), ('must', 0.9994497299194336), ('lays', 0.9994449615478516), ('awful', 0.9994432330131531), ('simply', 0.9994065761566162), ('comes', 0.9994044899940491), ('dinner', 0.9993829727172852)]


In [ ]:
w2v_words = list(w2v_model.wv.vocab)
print("number of words that occured minimum 5 times ",len(w2v_words))
print("sample words ", w2v_words[0:50])

number of words that occured minimum 5 times  3817
sample words  ['product', 'available', 'course', 'total', 'pretty', 'stinky', 'right', 'nearby', 'used', 'ca', 'not', 'beat', 'great', 'received', 'shipment', 'could', 'hardly', 'wait', 'try', 'love', 'call', 'instead', 'removed', 'easily', 'daughter', 'designed', 'printed', 'use', 'car', 'windows', 'beautifully', 'shop', 'program', 'going', 'lot', 'fun', 'everywhere', 'like', 'tv', 'computer', 'really', 'good', 'idea', 'final', 'outstanding', 'window', 'everybody', 'asks', 'bought', 'made']


In [ ]:
vector = w2v_model.wv['great'] 
vector

array([ 0.10668813,  0.21855555, -0.18562433, -0.3801391 , -0.0225658 ,
        0.02863738,  0.7448763 , -0.512877  ,  0.5538712 ,  0.4747185 ,
       -0.21609443,  0.17407696,  0.8822103 ,  0.832505  , -0.04136579,
        0.1464689 , -0.48537385,  0.19228749,  0.02275934, -0.49435344,
        0.6834954 ,  0.446906  , -0.8620672 , -0.64910924,  0.24169676,
       -0.22269964,  0.19491842, -0.4723902 ,  0.4702999 , -0.22891183,
        0.0357127 ,  0.4816934 , -0.2614937 , -0.23059505,  0.43415418,
        0.9828724 ,  0.02871656, -0.15503767, -0.4313076 ,  0.11662707,
       -0.90762687, -0.05392799, -0.6181212 , -0.16885221,  0.13210218,
       -0.34389526, -0.18700483,  0.36356977,  0.20211643,  0.3377593 ],
      dtype=float32)

In [ ]:
# Store just the words + their trained embeddings.
word_vectors = w2v_model.wv
word_vectors.save("word2vec")
# Load back with memory-mapping = read-only, shared across processes.
wv = KeyedVectors.load("word2vec", mmap='r')

vector = wv['great']  # Get numpy vector of a word
vector

array([ 0.10668813,  0.21855555, -0.18562433, -0.3801391 , -0.0225658 ,
        0.02863738,  0.7448763 , -0.512877  ,  0.5538712 ,  0.4747185 ,
       -0.21609443,  0.17407696,  0.8822103 ,  0.832505  , -0.04136579,
        0.1464689 , -0.48537385,  0.19228749,  0.02275934, -0.49435344,
        0.6834954 ,  0.446906  , -0.8620672 , -0.64910924,  0.24169676,
       -0.22269964,  0.19491842, -0.4723902 ,  0.4702999 , -0.22891183,
        0.0357127 ,  0.4816934 , -0.2614937 , -0.23059505,  0.43415418,
        0.9828724 ,  0.02871656, -0.15503767, -0.4313076 ,  0.11662707,
       -0.90762687, -0.05392799, -0.6181212 , -0.16885221,  0.13210218,
       -0.34389526, -0.18700483,  0.36356977,  0.20211643,  0.3377593 ],
      dtype=float32)

# Avg W2v

In [ ]:
# average Word2Vec
# compute average word2vec for each review.
sent_vectors = []; # the avg-w2v for each sentence/review is stored in this list
for sent in tqdm(list_of_sentance): # for each review/sentence
    sent_vec = np.zeros(50) # as word vectors are of zero length 50, you might need to change this to 300 if you use google's w2v
    cnt_words =0; # num of words with a valid vector in the sentence/review
    for word in sent: # for each word in a review/sentence
        if word in w2v_words:
            vec = w2v_model.wv[word]
            sent_vec += vec
            cnt_words += 1
    if cnt_words != 0:
        sent_vec /= cnt_words
    sent_vectors.append(sent_vec)
print(len(sent_vectors))
print(len(sent_vectors[0]))

100%|█████████████████████████████████████████████████████████████████████████████| 4986/4986 [00:05<00:00, 971.75it/s]


4986
50


# TFIDF weighted W2v

In [ ]:
# S = ["abc def pqr", "def def def abc", "pqr pqr def"]
model = TfidfVectorizer()
model.fit(preprocessed_reviews)
# we are converting a dictionary with word as a key, and the idf as a value
dictionary = dict(zip(model.get_feature_names(), list(model.idf_)))

In [ ]:
# TF-IDF weighted Word2Vec
tfidf_feat = model.get_feature_names() # tfidf words/col-names
# final_tf_idf is the sparse matrix with row= sentence, col=word and cell_val = tfidf

tfidf_sent_vectors = []; # the tfidf-w2v for each sentence/review is stored in this list
row=0;
for sent in tqdm(list_of_sentance): # for each review/sentence 
    sent_vec = np.zeros(50) # as word vectors are of zero length
    weight_sum =0; # num of words with a valid vector in the sentence/review
    for word in sent: # for each word in a review/sentence
        if word in w2v_words and word in tfidf_feat:
            vec = w2v_model.wv[word]
#             tf_idf = tf_idf_matrix[row, tfidf_feat.index(word)]
            # to reduce the computation we are 
            # dictionary[word] = idf value of word in whole courpus
            # sent.count(word) = tf valeus of word in this review
            tf_idf = dictionary[word]*(sent.count(word)/len(sent))
            sent_vec += (vec * tf_idf)
            weight_sum += tf_idf
    if weight_sum != 0:
        sent_vec /= weight_sum
    tfidf_sent_vectors.append(sent_vec)
    row += 1

100%|█████████████████████████████████████████████████████████████████████████████| 4986/4986 [00:27<00:00, 179.19it/s]


In [ ]:
# text preprocessing: two diff methods

import nltk                   # natural lang processing tool kit
import re                     # regular expressions

nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')

# tokenization - sentences/words: break up
# sample paragraph
p = 'my name is Moni. I live with my family. I like my family very much'

sentences = nltk.sent_tokenize(p)
print(sentences)
w = nltk.word_tokenize(p)
print(w)

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\Chara\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Chara\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\Chara\AppData\Roaming\nltk_data...


['my name is Moni.', 'I live with my family.', 'I like my family very much']
['my', 'name', 'is', 'Moni', '.', 'I', 'live', 'with', 'my', 'family', '.', 'I', 'like', 'my', 'family', 'very', 'much']


[nltk_data]   Package wordnet is already up-to-date!


In [ ]:
# Stemming - porterstemmer/snowballstemmer - find root/stem of words but not much useful
# as there  is no meaning n takes less time..eg: spam detection
# Stopwords - [is,was,he,she]-- doesnt give much info

from nltk.stem import SnowballStemmer
from nltk.corpus import stopwords

stemmer = SnowballStemmer('english')    # initializing

for i in range(len(sentences)):
    words = nltk.word_tokenize(sentences[i])
    words = [stemmer.stem(j) for j in words if  j not in  set(stopwords.words('english'))]
    sentences[i]= ' '.join(words)
    
sentences

['name moni .', 'i live famili .', 'i like famili much']

In [ ]:
# lemmatization - better than stemming as it is meaningful ; eg: siri(Q-A appln)

from nltk.stem import WordNetLemmatizer

lemmatizer = WordNetLemmatizer()

for i in range(len(sentences)):
    words = nltk.word_tokenize(sentences[i])
    words = [lemmatizer.lemmatize(j) for j in words if  j not in  set(stopwords.words('english'))]
    sentences[i]= ' '.join(words)
    
sentences

['name moni .', 'live famili .', 'like famili much']

In [ ]:
# Bag of words: converts text to vector; doesnt preserve semantic meaning; sparse matrix(many 0's)

from sklearn.feature_extraction.text import CountVectorizer

p1 = 'my name is Moni''. I live with my family. </>I like my family very much'
sentences1 = nltk.sent_tokenize(p1)
corpus = []
for i in range(len(sentences1)):
    data_c = re.sub('[^a-zA-Z]',' ',sentences1[i])  # removes all extra char or quotes..only keep a-z/ A-Z
    #print(data_c)
    data_c = data_c.lower()
    data_c= data_c.split()
    data_c = [lemmatizer.lemmatize(j) for j in data_c if  j not in  set(stopwords.words('english'))]
    data_c= ' '.join(data_c)
    corpus.append(data_c)
#print(corpus)
# BOW model:    
    
cv = CountVectorizer()
x = cv.fit_transform(corpus).toarray()
x

array([[0, 0, 0, 1, 0, 1],
       [1, 0, 1, 0, 0, 0],
       [1, 1, 0, 0, 1, 0]], dtype=int64)

In [ ]:
# TF-IDF Model:  converts text to vector ; doesnt preserve semantic relationship b/w words

from sklearn.feature_extraction.text import TfidfVectorizer

p1 = 'my name is Moni''. I live with my family. </>I like my family very much'
sentences1 = nltk.sent_tokenize(p1)
corpus = []
for i in range(len(sentences1)):
    data_c = re.sub('[^a-zA-Z]',' ',sentences1[i])  # removes all extra char or quotes..only keep a-z/ A-Z
    #print(data_c)
    data_c = data_c.lower()
    data_c= data_c.split()
    data_c = [lemmatizer.lemmatize(j) for j in data_c if  j not in  set(stopwords.words('english'))]
    data_c= ' '.join(data_c)
    corpus.append(data_c)
#print(corpus)
# TF-IDF model:    
    
tfidf = TfidfVectorizer()
x = tfidf.fit_transform(corpus).toarray()
x


array([[0.        , 0.        , 0.        , 0.70710678, 0.        ,
        0.70710678],
       [0.60534851, 0.        , 0.79596054, 0.        , 0.        ,
        0.        ],
       [0.4736296 , 0.62276601, 0.        , 0.        , 0.62276601,
        0.        ]])